In [30]:
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
% matplotlib inline

# Preprocessing of data

In [ ]:
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

data_train.head(5)

data_train.shape

data_test.head(5)

data_test.shape

data_train['Day'] = data_train['Open Date'].apply(lambda x: int(x[:2]))
data_train['Month'] = data_train['Open Date'].apply(lambda x: int(x[3:5]))
data_train['Year'] = data_train['Open Date'].apply(lambda x: int(x[6:]))

del data_train['Open Date']

data_train.head(5)

y_train = data_train['revenue'].copy()

X_train = data_train.copy()
del X_train['revenue']

X_train.head()

y_train.head(5)

data_test['Day'] = data_test['Open Date'].apply(lambda x: int(x[:2]))
data_test['Month'] = data_test['Open Date'].apply(lambda x: int(x[3:5]))
data_test['Year'] = data_test['Open Date'].apply(lambda x: int(x[6:]))

del data_test['Open Date']

data_test.head()

X_test = data_test.copy()

del X_train['City']
del X_test['City']

del X_train['City Group']
del X_test['City Group']

X_train["TypeInt"] = X_train.Type.map({'FC': 0, 'IL': 1, 'DT': 2, 'MB': 3})
X_test["TypeInt"] = X_test.Type.map({'FC': 0, 'IL': 1, 'DT': 2, 'MB': 3})

del X_train["Type"]

In [3]:
del X_test["Type"]

# TPOT

In [26]:
from tpot import TPOTRegressor

In [28]:
model = TPOTRegressor(generations=10, population_size=30, 
                      offspring_size=10, cv = 5, 
                      n_jobs = 1, scoring = 'neg_mean_squared_error',
                      verbosity = 4)

29 operators have been imported by TPOT.


In [46]:
start_time = time.time()
model.fit(X_train, y_train)
print('Training time: ', time.time() - start_time)

Optimization Progress:  38%|███▊      | 50/130 [00:15<00:33,  2.42pipeline/s]

_pre_test decorator: _random_mutation_operator: num_test=0 __init__() got an unexpected keyword argument 'max_depth'
_pre_test decorator: _random_mutation_operator: num_test=0 __init__() got an unexpected keyword argument 'max_depth'


Optimization Progress:  55%|█████▍    | 71/130 [00:28<00:47,  1.23pipeline/s]

Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.


Optimization Progress:  85%|████████▍ | 110/130 [01:00<00:20,  1.04s/pipeline]

_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False


Training time:  78.25010085105896


In [47]:
predictions = model.predict(X_test)

In [48]:
predictions

array([5156722.32216669, 4652795.52006888, 3438344.74925425, ...,
       4666090.32193781, 3894997.94631687, 4636119.8256163 ])

In [50]:
output = pd.DataFrame({'Id': data_test['Id'], 'Prediction': predictions})
output.to_csv('tpot_1.csv', index = False)